In [16]:
import requests

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('jIr-nVzfumFqGw', 'ZHqDRjAawvPJKdHsfUwrpS57LDtiBQ')

# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'jpz129',
        'password': 'IloveRhiannon129!@('}

# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'MyBot/0.0.1'}

# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)

# convert response to JSON and pull access_token value
TOKEN = res.json()['access_token']

# add authorization to our headers dictionary
headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# while the token is valid (~2 hours) we just add headers=headers to our requests
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)

<Response [200]>

In [17]:
res = requests.get("https://oauth.reddit.com/r/seafood/hot",
                   headers=headers,
                   params={'limit': '100'})

print(res.json())  # let's see what we get

{'kind': 'Listing', 'data': {'modhash': None, 'dist': 100, 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'Seafood', 'selftext': '', 'author_fullname': 't2_xb67b', 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Buttermilk fried sea scallops for my fathers birthday! Made them not long ago and they were delicious.', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/Seafood', 'hidden': False, 'pwls': 6, 'link_flair_css_class': None, 'downs': 0, 'thumbnail_height': 140, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_m55j7c', 'quarantine': False, 'link_flair_text_color': 'dark', 'upvote_ratio': 0.96, 'author_flair_background_color': None, 'subreddit_type': 'public', 'ups': 30, 'total_awards_received': 1, 'media_embed': {}, 'thumbnail_width': 140, 'author_flair_template_id': None, 'is_original_content': False, 'user_reports': [], 'secure_media': None, 'is_reddit_media_domain': True, 'is_meta': False, 'category

In [18]:
import pandas as pd
# make a request for the trending posts in /r/Python
res = requests.get("https://oauth.reddit.com/r/seafood",
                   headers=headers)

df = pd.DataFrame()  # initialize dataframe

# loop through each post retrieved from GET request
for post in res.json()['data']['children']:
    # append relevant data to dataframe
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score']
    }, ignore_index=True)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   downs         25 non-null     float64
 1   score         25 non-null     float64
 2   selftext      25 non-null     object 
 3   subreddit     25 non-null     object 
 4   title         25 non-null     object 
 5   ups           25 non-null     float64
 6   upvote_ratio  25 non-null     float64
dtypes: float64(4), object(3)
memory usage: 1.5+ KB


In [20]:
df.head()

,downs,score,selftext,subreddit,title,ups,upvote_ratio
0,0.0,29.0,,Seafood,Buttermilk fried sea scallops for my fathers b...,29.0,0.96
1,0.0,52.0,,Seafood,Scallops,52.0,0.95
2,0.0,3.0,,Seafood,How To Make | Maryland Style | Crab Cakes | Wi...,3.0,1.00
3,0.0,0.0,I know calamari can become tender again with l...,Seafood,Can over cooked lobster be saved?,0.0,0.50
4,0.0,58.0,,Seafood,Steamed mussels with white wine and garlic,58.0,1.00


In [21]:
# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df

In [43]:
# initialize dataframe and parameters for pulling data in loop
data = pd.DataFrame()
params = {'limit': 100}

# loop through 10 times (returning 1K posts)
for i in range(1):
    # make request
    res = requests.get("https://oauth.reddit.com/r/seafood/new",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   created_utc           100 non-null    object 
 1   downs                 100 non-null    float64
 2   id                    100 non-null    object 
 3   kind                  100 non-null    object 
 4   link_flair_css_class  0 non-null      object 
 5   score                 100 non-null    float64
 6   selftext              100 non-null    object 
 7   subreddit             100 non-null    object 
 8   title                 100 non-null    object 
 9   ups                   100 non-null    float64
 10  upvote_ratio          100 non-null    float64
dtypes: float64(4), object(7)
memory usage: 8.7+ KB


In [45]:
data.head()

,created_utc,downs,id,kind,link_flair_css_class,score,selftext,subreddit,title,ups,upvote_ratio
0,2021-03-14T23:05:19Z,0.0,m5bag7,t3,None,0.0,I know calamari can become tender again with l...,Seafood,Can over cooked lobster be saved?,0.0,0.50
1,2021-03-14T18:16:47Z,0.0,m55ris,t3,None,3.0,,Seafood,How To Make | Maryland Style | Crab Cakes | Wi...,3.0,1.00
2,2021-03-14T18:05:49Z,0.0,m55j7c,t3,None,32.0,,Seafood,Buttermilk fried sea scallops for my fathers b...,32.0,0.98
3,2021-03-14T12:48:57Z,0.0,m4yqca,t3,None,54.0,,Seafood,Scallops,54.0,0.97
4,2021-03-14T00:18:35Z,0.0,m4op0b,t3,None,0.0,Where to order king crab legs online to delive...,Seafood,Where to buy king crab legs ?,0.0,0.33


In [42]:
data.iloc[100]

created_utc                                          2021-02-11T17:41:27Z
downs                                                                   0
id                                                                 lhwx2t
kind                                                                   t3
link_flair_css_class                                                 None
score                                                                   5
selftext                                                              🤩🤩🤩
subreddit                                                         Seafood
title                   IM GETTING TO MAKE A SEAFOOD BOIL TOMORROW AND...
ups                                                                     5
upvote_ratio                                                            1
Name: 100, dtype: object

In [49]:
import requests
import pandas as pd
from datetime import datetime

# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df

# authenticate API
client_auth = requests.auth.HTTPBasicAuth('jIr-nVzfumFqGw', 'ZHqDRjAawvPJKdHsfUwrpS57LDtiBQ')
data = {
    'grant_type': 'password',
    'username': 'jpz129',
    'password': 'IloveRhiannon129!@('
}
headers = {'User-Agent': 'myBot/0.0.1'}

# send authentication request for OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=data, headers=headers)
# extract token from response and format correctly
TOKEN = f"bearer {res.json()['access_token']}"
# update API headers with authorization (bearer token)
headers = {**headers, **{'Authorization': TOKEN}}

# initialize dataframe and parameters for pulling data in loop
data = pd.DataFrame()
params = {'limit': 100}

# loop through 10 times (returning 1K posts)
for i in range(3):
    # make request
    res = requests.get("https://oauth.reddit.com/r/python/new",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    # take the final row (oldest entry)
    row = new_df.iloc[len(new_df)-1]
    # create fullname
    fullname = row['kind'] + '_' + row['id']
    # add/update fullname in params
    params['after'] = fullname
    
    # append new_df to data
    data = data.append(new_df, ignore_index=True)

In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   created_utc           300 non-null    object 
 1   downs                 300 non-null    float64
 2   id                    300 non-null    object 
 3   kind                  300 non-null    object 
 4   link_flair_css_class  300 non-null    object 
 5   score                 300 non-null    float64
 6   selftext              300 non-null    object 
 7   subreddit             300 non-null    object 
 8   title                 300 non-null    object 
 9   ups                   300 non-null    float64
 10  upvote_ratio          300 non-null    float64
dtypes: float64(4), object(7)
memory usage: 25.9+ KB


In [53]:
data.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   created_utc           300 non-null    object 
 1   downs                 300 non-null    float64
 2   id                    300 non-null    object 
 3   kind                  300 non-null    object 
 4   link_flair_css_class  300 non-null    object 
 5   score                 300 non-null    float64
 6   selftext              300 non-null    object 
 7   subreddit             300 non-null    object 
 8   title                 300 non-null    object 
 9   ups                   300 non-null    float64
 10  upvote_ratio          300 non-null    float64
dtypes: float64(4), object(7)
memory usage: 28.1+ KB
